# Conducting a Raffle Using Marlowe


**Executive Summary**

This example demonstrates conducting a raffle using Marlowe. The contract contains a list of addresses of participants who have entered the raffle, and a random oracle selects and pays one of them. The Blockly diagram below shows a simple raffle with three participants. For large numbers of participants, multiple nested `When` statements are used to hierarchically decompose the search for the winner, so that the Plutus execution budget is not exceeded by the Marlowe contract. Instead of awarding to specific addresses, this example could trivially be modified to award to the holders of a sequence of role tokens.

![Marlowe contract for a raffle](contract.png)

A [video demonstration of this example](https://youtu.be/wtUFkElA-u0) is available.

## Program for Generating the Contract

Create a Haskell program that will generate a contract for the given participants..

The program outputs the file [contract.json](contract.json).

In [2]:
cat << EOI > contract.hs

{-# LANGUAGE NumericUnderscores #-}
{-# LANGUAGE OverloadedStrings  #-}
{-# LANGUAGE Trustworthy        #-}

module Main (
-- * Entry point
  main
-- * Contracts
, makeContract
) where

import Data.Aeson ((.=), encodeFile)
import Data.List.Split (chunksOf)
import Data.Maybe (fromJust)
import Data.String (fromString)
import Data.Text (pack)
import Language.Marlowe.Core.V1.Semantics.Types
import Language.Marlowe.Core.V1.Semantics.Types.Address (deserialiseAddressBech32)
import Plutus.V2.Ledger.Api (POSIXTime(..), TokenName(..))
import System.Environment (getArgs)

-- | Print the contract.
main :: IO ()
main =
  do
    size : sponsor : oracle : amount : deposit : select : payout : parties <- getArgs
    let
      readAddress = uncurry Address . fromJust . deserialiseAddressBech32 . pack
      contract =
        makeContract
          (readAddress sponsor)
          (readAddress oracle)
          (read amount)
          (read size)
          (zip [1..] $ readAddress <$> parties)
          (fromInteger $ read deposit)
          (fromInteger $ read select)
          (fromInteger $ read payout)
    encodeFile "contract.json" contract

makeContract
  :: Party
  -> Party
  -> Integer
  -> Int
  -> [(Integer, Party)]
  -> POSIXTime
  -> POSIXTime
  -> POSIXTime
  -> Contract
makeContract sponsor oracle amount size parties deposit select payout =
  makeDeposit sponsor amount deposit
    $ selectWinner oracle (toInteger $ length parties) select
    $ payWinner sponsor oracle amount size parties payout
    
ada :: Token
ada = Token "" ""

makeDeposit
  :: Party
  -> Integer
  -> POSIXTime
  -> Contract
  -> Contract
makeDeposit sponsor amount deadline contract =
  When
    [Case (Deposit sponsor sponsor ada (Constant amount)) contract]
    deadline
    Close
    
selectWinner
  :: Party
  -> Integer
  -> POSIXTime
  -> Contract
  -> Contract
selectWinner oracle count deadline contract =
  When
    [Case (Choice (ChoiceId "Random" oracle) [Bound 1 count]) contract]
    deadline
    Close

payWinner
  :: Party
  -> Party
  -> Integer
  -> Int
  -> [(Integer, Party)]
  -> POSIXTime
  -> Contract
payWinner sponsor oracle amount size parties deadline
  | length parties <= size = When
                               [
                                 Case (Notify (ValueEQ (ChoiceValue (ChoiceId "Random" oracle)) (Constant i)))
                                   $ Pay sponsor (Party party) ada (Constant amount) Close
                               |
                                 (i, party) <- parties
                               ]
                               deadline
                               Close
  | otherwise              = When
                               [
                                 Case (Notify (ValueLE (ChoiceValue (ChoiceId "Random" oracle)) (Constant . fst $ last parties')))
                                   $ payWinner sponsor oracle amount size parties' deadline
                               |
                                 let chunks k xs = chunksOf (div (length xs + k - 1) k) xs
                               , parties' <- chunks size parties
                               ]
                               deadline
                               Close

EOI

## Constants

It is convenient to define serveral constants.

In [3]:
ADA=1000000

In [4]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))

## Network

Use the preproduction network.

In [5]:
export CARDANO_NODE_SOCKET_PATH=/extra/iohk/networks/preprod/node.socket
export CARDANO_TESTNET_MAGIC=1

## Parties to the contract

Set the signing keys and addresses for parties to the contract.

### Sponsor

The *Sponsor* provides the funds for the raffle.

In [6]:
SPONSOR_NAME=c.marlowe
SPONSOR_SKEY=../keys/$SPONSOR_NAME.skey
SPONSOR_ADDR=$(cat ../keys/$SPONSOR_NAME.testnet.address)
echo "$SPONSOR_NAME "@" $SPONSOR_ADDR"

c.marlowe @ addr_test1vrtntkszteptml4e9ce9l3fsmgavwv4ywunvdnhxv6nw5ksq6737a


### Oracle

The *Oracle* provides the random number for the raffle.

In [7]:
ORACLE_NAME=f.beaumont
ORACLE_SKEY=../keys/$ORACLE_NAME.skey
ORACLE_ADDR=$(cat ../keys/$ORACLE_NAME.testnet.address)
echo "$ORACLE_NAME" @ "$ORACLE_ADDR"

f.beaumont @ addr_test1vqhqudxtwqcpjqesns79hqgqq2q0xx5q0hnzz5es9492yaqpxltpy


### Participants

Normally, the list of addresses for the *Participants* would be provided. Here, for convenience, we just derive each participants's address from the same root private key.

In [8]:
ROOT_PRV=../keys/william-shakespeare.root.prv

Also for convenience, the scholars will share the same stake key.

In [9]:
cardano-wallet key child 1852H/1815H/0H/2/0 < "$ROOT_PRV" \
| cardano-cli key convert-cardano-address-key --shelley-stake-key --signing-key-file /dev/stdin --out-file /dev/stdout \
| cardano-cli key verification-key --signing-key-file /dev/stdin --verification-key-file /dev/stdout \
| cardano-cli key non-extended-key --extended-verification-key-file /dev/stdin --verification-key-file ../keys/william-shakespeare.stake.vkey
STAKE_ADDR=`cardano-cli stake-address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --stake-verification-key-file ../keys/william-shakespeare.stake.vkey`
echo "Stake address = $STAKE_ADDR"

Stake address = stake_test1urplvp2a7dythh6yxutd0qlzzkncr3gy5ftvxj02d3etafqjugc5h


Set the number of scholars.

In [10]:
NUMBER_OF_PARTICIPANTS=125

Compute the addresses of the scholars.

In [11]:
for j in `seq 1 $NUMBER_OF_PARTICIPANTS`
do
  PARTICIPANT_SKEY[$j]=../keys/scholar-$j.skey
  PARTICIPANT_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$j < "$ROOT_PRV" \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file "${PARTICIPANT_SKEY[$j]}"
  cardano-cli key verification-key --signing-key-file "${PARTICIPANT_SKEY[$j]}" --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file /dev/stdin --stake-verification-key-file ../keys/william-shakespeare.stake.vkey \
  )
  echo "Participant $j" = 1852H/1815H/0H/0/"$((1 + j))" = "${PARTICIPANT_ADDR[$j]}"
done

Participant 1 = 1852H/1815H/0H/0/2 = addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p
Participant 2 = 1852H/1815H/0H/0/3 = addr_test1qraxynufcduk7ak7nke2jm6pnc32vn7wsmm7ml30cutfgg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq77wfgp
Participant 3 = 1852H/1815H/0H/0/4 = addr_test1qzlwepx5u26a8w94kfrkfem6njlam22emyhjscu0lxqrce7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq5jnrn3
Participant 4 = 1852H/1815H/0H/0/5 = addr_test1qqjrjfslev50d5jn6qdeztxsqvuaaefq6e4hu55n59grjj7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqwhulyr
Participant 5 = 1852H/1815H/0H/0/6 = addr_test1qqxrzfv0v8l0lahyc5qsgpug66dafp8njl278g0pter7e3xr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqcjuuw0
Participant 6 = 1852H/1815H/0H/0/7 = addr_test1qqqfzyuqkvrr7ajgk887sqac8lgzatc6el9uxke4x5g5dg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqkmrpwu
Participant 7 = 1852H/1815H/0H/0/8 = addr_test1qp77epd4gy5hrt8pnh6h73se4durwrrkes3u0vj6tycwsq7r7cz4mu6gh005gdck67p7y9d8s8zsf

## Contract Parameters

### Deadlines

Start the contract at the current time.

In [12]:
NOW=$((`date -u +%s` * 1000))
echo "NOW = $NOW" = "`date -d @$((NOW / 1000))`"

NOW = 1683600635000 = Mon May  8 08:50:35 PM MDT 2023


In [13]:
FUNDING_DEADLINE=$((`date -u +%s` * SECOND + 1 * HOUR))
echo "FUNDING_DEADLINE = $FUNDING_DEADLINE" = "`date -d @$((FUNDING_DEADLINE / 1000))`"

FUNDING_DEADLINE = 1683604238000 = Mon May  8 09:50:38 PM MDT 2023


In [14]:
SELECTION_DEADLINE=$((`date -u +%s` * SECOND + 2 * HOUR))
echo "SELECTION_DEADLINE = $SELECTION_DEADLINE" = "`date -d @$((SELECTION_DEADLINE / 1000))`"

SELECTION_DEADLINE = 1683607840000 = Mon May  8 10:50:40 PM MDT 2023


In [15]:
AWARD_DEADLINE=$((`date -u +%s` * SECOND + 3 * HOUR))
echo "AWARD_DEADLINE = $AWARD_DEADLINE" = "`date -d @$((AWARD_DEADLINE / 1000))`"

AWARD_DEADLINE = 1683611444000 = Mon May  8 11:50:44 PM MDT 2023


### Award

In [16]:
AWARD_AMOUNT=$((100 * ADA))
echo "AWARD_AMOUNT = $AWARD_AMOUNT lovelace"

AWARD_AMOUNT = 100000000 lovelace


## Contract

Now run the Haskell program to generate the contract. We'll use a fivefold split to hierarchically search for the winner, so that the Plutus execution budget is not exceeded.

In [17]:
SIZE=5
echo "SIZE = $SIZE"

SIZE = 5


In [18]:
runhaskell contract.hs \
  "$SIZE" \
  "$SPONSOR_ADDR" \
  "$ORACLE_ADDR" \
  "$AWARD_AMOUNT" \
  "$FUNDING_DEADLINE" \
  "$SELECTION_DEADLINE" \
  "$AWARD_DEADLINE" \
  ${PARTICIPANT_ADDR[@]}

## Initial State

We put 2 ada in the initial contract when it is created, in order to satisfy the ledger's minimum-UTxO rule.

In [19]:
MIN_LOVELACE=$((2 * ADA))

yaml2json << EOI > state.json
accounts:
- - - address: $SPONSOR_ADDR
    - currency_symbol: ''
      token_name: ''
  - $MIN_LOVELACE
boundValues: []
choices: []
minTime: 1
EOI

cat state.json

{"accounts":[[[{"address":"addr_test1vrtntkszteptml4e9ce9l3fsmgavwv4ywunvdnhxv6nw5ksq6737a"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":1}


## Creation transaction

Merkleize the contract when we initialize it.

In [20]:
marlowe-cli run initialize \
  --contract-file contract.json \
  --state-file state.json \
  --out-file marlowe-1.json \
  --merkleize \
  --permanently-without-staking

The sponsor submits the creation transaction.

In [21]:
TX[1]=$(
/extra/iohk/bin/marlowe-cli run auto-execute \
  --marlowe-out-file marlowe-1.json \
  --change-address "$SPONSOR_ADDR" \
  --required-signer "$SPONSOR_SKEY" \
  --out-file /dev/null \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX[1] = ${TX[1]}"
echo 'https://preprod.cardanoscan.io/transaction/'"${TX[1]}"'?tab=utxo'


Fee: Lovelace 196081
Size: 579 / 16384 = 3%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX[1] = c5f1307a4f2d12b419b1ed381a8ccefcf4c2a019c9e2f663da24cb3a91258206
https://preprod.cardanoscan.io/transaction/c5f1307a4f2d12b419b1ed381a8ccefcf4c2a019c9e2f663da24cb3a91258206?tab=utxo


## Transaction to deposit the funds.

The sponsor prepares the transaction to deposit the funds.

In [22]:
marlowe-cli run prepare \
  --deposit-account "$SPONSOR_ADDR" \
  --deposit-party "$SPONSOR_ADDR" \
  --deposit-amount "$AWARD_AMOUNT" \
  --invalid-before "$((($(date -u +%s) - 1 * 60) * 1000))" \
  --invalid-hereafter "$((($(date -u +%s) + 5 * 60) * 1000))" \
  --marlowe-file marlowe-1.json \
  --out-file marlowe-2.json \
  2> /dev/null

TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1683600666000},POSIXTime {getPOSIXTime = 1683601026999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrtntkszteptml4e9ce9l3fsmgavwv4ywunvdnhxv6nw5ksq6737a\"" "\"addr_test1vrtntkszteptml4e9ce9l3fsmgavwv4ywunvdnhxv6nw5ksq6737a\"" (Token "" "") 100000000)]}


Submit the transaction and await its confirmation.

In [23]:
TX[2]=$(
/extra/iohk/bin/marlowe-cli run auto-execute \
  --tx-in-marlowe "${TX[1]}#1" \
  --marlowe-in-file marlowe-1.json \
  --marlowe-out-file marlowe-2.json \
  --change-address "$SPONSOR_ADDR" \
  --required-signer "$SPONSOR_SKEY" \
  --out-file /dev/null \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX[2] = ${TX[2]}"
echo 'https://preprod.cardanoscan.io/transaction/'"${TX[2]}"'?tab=utxo'


Fee: Lovelace 625267
Size: 1250 / 16384 = 7%
Execution units:
  Memory: 5168716 / 14000000 = 36%
  Steps: 1405524214 / 10000000000 = 14%
TX[2] = 3c832480473eedcbb911855be6e9f434d31ccb91daa087ee162c6a5c9d8b28c3
https://preprod.cardanoscan.io/transaction/3c832480473eedcbb911855be6e9f434d31ccb91daa087ee162c6a5c9d8b28c3?tab=utxo


## Transaction to randomly select a winner.

The oracle fetches a truly random integer from [RANDOM.ORG](https://random.org/).

In [25]:
SELECTION=$(curl -sS "https://www.random.org/integers/?num=1&min=1&max="$NUMBER_OF_PARTICIPANTS"&col=1&base=10&format=plain&rnd=new")
echo "SELECTION = $SELECTION"

SELECTION = 85


The oracle prepares the transaction with this random choice.

In [26]:
marlowe-cli run prepare \
  --choice-name Random \
  --choice-party "$ORACLE_ADDR" \
  --choice-number "$SELECTION" \
  --invalid-before "$((($(date -u +%s) - 1 * 60) * 1000))" \
  --invalid-hereafter "$((($(date -u +%s) + 5 * 60) * 1000))" \
  --marlowe-file marlowe-2.json \
  --out-file marlowe-3.json \
  2> /dev/null

TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1683600721000},POSIXTime {getPOSIXTime = 1683601081999}), txInputs = [NormalInput (IChoice (ChoiceId "Random" "\"addr_test1vqhqudxtwqcpjqesns79hqgqq2q0xx5q0hnzz5es9492yaqpxltpy\"") 85)]}


The oracle submits the transaction and awaits confirmation.

In [27]:
TX[3]=$(
/extra/iohk/bin/marlowe-cli run auto-execute \
  --tx-in-marlowe "${TX[2]}#1" \
  --marlowe-in-file marlowe-2.json \
  --marlowe-out-file marlowe-3.json \
  --change-address "$ORACLE_ADDR" \
  --required-signer "$ORACLE_SKEY" \
  --out-file /dev/null \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX[3] = ${TX[3]}"
echo 'https://preprod.cardanoscan.io/transaction/'"${TX[3]}"'?tab=utxo'


Fee: Lovelace 833385
Size: 2172 / 16384 = 13%
Execution units:
  Memory: 7171896 / 14000000 = 51%
  Steps: 2123841881 / 10000000000 = 21%
TX[3] = 4d0f2f250203ca39074859ced409f1e4d8b6a2ef81f780c1f21b6920a4d156e4
https://preprod.cardanoscan.io/transaction/4d0f2f250203ca39074859ced409f1e4d8b6a2ef81f780c1f21b6920a4d156e4?tab=utxo


## Transactions to award the winner

Because the search for the winner has been hierarchically split, we'll need several `INotify` actions to walk the hierarchy to the winner. The number of notifications is the logarithm of the number of participants, rounded upwards to the nearest whole number.

In [28]:
DEPTH=$(printf "%.0f" `echo l"($NUMBER_OF_PARTICIPANTS)" / "l($SIZE)" | ~/.nix-profile/bin/bc -l`)
echo "DEPTH = $DEPTH"

DEPTH = 3


Anyone can prepare and submit the notifications, but for convenience the sponsor does so here.

In [29]:
for i in `seq 1 $DEPTH`
do

n=$((i+3))

echo
echo "----- Notify $i -----"
echo

marlowe-cli run prepare \
  --notify \
  --invalid-before "$((($(date -u +%s) - 1 * 60) * 1000))" \
  --invalid-hereafter "$((($(date -u +%s) + 5 * 60) * 1000))" \
  --marlowe-file marlowe-$((n-1)).json \
  --out-file marlowe-$n.json \
  2> /dev/null

TX[$n]=$(
/extra/iohk/bin/marlowe-cli run auto-execute \
  --tx-in-marlowe "${TX[$((n-1))]}#1" \
  --marlowe-in-file marlowe-$((n-1)).json \
  --marlowe-out-file marlowe-$n.json \
  --change-address "$SPONSOR_ADDR" \
  --required-signer "$SPONSOR_SKEY" \
  --out-file /dev/null \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX[$n] = ${TX[$n]}"
echo 'https://preprod.cardanoscan.io/transaction/'"${TX[$n]}"'?tab=utxo'  

done


----- Notify 1 -----

TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1683600792000},POSIXTime {getPOSIXTime = 1683601152999}), txInputs = [NormalInput INotify]}

Fee: Lovelace 913392
Size: 2654 / 16384 = 16%
Execution units:
  Memory: 7896032 / 14000000 = 56%
  Steps: 2362301577 / 10000000000 = 23%
TX[4] = 39981f87fabe70091285af176b8a97b234cc16c6ffde4b948fa39282070af861
https://preprod.cardanoscan.io/transaction/39981f87fabe70091285af176b8a97b234cc16c6ffde4b948fa39282070af861?tab=utxo

----- Notify 2 -----

TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1683600803000},POSIXTime {getPOSIXTime = 1683601163999}), txInputs = [NormalInput INotify]}

Fee: Lovelace 896647
Size: 2654 / 16384 = 16%
Execution units:
  Memory: 7679900 / 14000000 = 54%
  Steps: 2303018611 / 10000000000 = 23%
TX[5] = 39ee58ea4d271c29f7c529e0730c8b22acd69fba58f2bd819b1c3ae310b1e4fc
https://preprod.cardanoscan.io/transaction/39ee58ea4d271c29f7c529e0730c8b22acd69fba58f2bd819b1c3ae310b1e4fc?tab=u

The last notification triggered the payment to the winner and a reimbursement of the sponsor's minimum-UTxO value from creating the contract.

In [30]:
echo "Winner = ${PARTICIPANT_ADDR[$SELECTION]}"

Winner = addr_test1qrcnznq5m5srjasmxrpkdqc7x4mh02vsru0ws8m82v667hxr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq8wje78


Here is the UTxO with the winnings.

In [31]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${PARTICIPANT_ADDR[$SELECTION]}"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
296d39e690bf0e8d6bf65075b8fe4ce0efd7b1acc2471edd1c32b3b906d5f824     2        100000000 lovelace + TxOutDatumNone
